![license_header_logo](https://user-images.githubusercontent.com/59526258/124226124-27125b80-db3b-11eb-8ba1-488d88018ebb.png)
> **Copyright (c) 2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.
<br>

## Introduction
In this tutorial, we will learn on how to automate data validation by using some of the pre-built packages.<br>

Spending multiple hour a day and figure out that data came through was incorrect for whatever reasons can be relieving but frustrating at the same time. There are many reasons to have incorrect data such this as text introduced as integer, an integer is probably an outlier or worse-case scenario if entire specific column was not received in the data feed.<br>

At the end of this tutorial, you should be able to:
1. Automate data validation in your project
2. Able to handle issues regarding data type, data format and etc.
3. Detect which of column of your dataset give errors

## Data Validation Packages : Pandera
`pandera` is a statistical data validation library for `pandas` data structures. `pandera`provides a versatile and expressive API for performing data validation on tidy (long-form) and wide data to make data processing pipelines more readable and robust.<br>

More information can be retrieved <a href=https://pandera.readthedocs.io/en/stable/>here</a>.
## Table of Content
* [Data Type and Data Format](#typeformat)
* [Data Duplication](#duplicate)
* [Exercise ](#exercise)

In [ ]:
#import libraries
import pandas as pd
import pandera as pa
from pandera.errors import SchemaError
import numpy as np

In [ ]:
# Create dataframe
df = pd.DataFrame({
    "Student ID": [101, 102, 103, 104, 105, 106, 107, 107, 108, 109],
    "Gender" : ["F", np.nan, "M", "M", np.nan, "M", "F", "F", "M", "F"],
    "Grade" : [54, 85, -15, 20, 60, 96, 84, 84, -25, 17],
    "Student fee" : [1320.0, 1450.0, 1200.0, 3200.50, 2500.0, 1785.5, 3100.2, 3100.2, 1540.0, 1630.0],
    "Date joined": pd.to_datetime(["12/01/2021", "02/01/2021", "02/02/2021", "07/02/2021", "28/12/2020", 
                    "07/01/2021", "20/11/2019", "20/11/2019", "01/02/2021", "23/01/2021"], format="%d/%m/%Y")
})

In [ ]:
df.head(10)

In [ ]:
df.dtypes

## <a name="typeformat">Data Type and Data Format
Here we check if our data format fit with the schema that we provided below.

In [ ]:
# Define schema
schema = pa.DataFrameSchema({
    "Student ID" : pa.Column(int, nullable=False),
    "Gender" : pa.Column(str, checks=pa.Check(lambda x: x.isin(["M", "F"])), nullable=True),
    "Grade" : pa.Column(int, checks=pa.Check.greater_than(0)),
    "Student fee" : pa.Column(float, checks=pa.Check.in_range(0.0, 3500.00)),
    "Date joined" : pa.Column(pa.DateTime, checks=pa.Check.greater_than_or_equal_to("2021-01-01"))
})

In [ ]:
schema.validate(df, lazy=True)

<b>lazy=True</b> is used to display in details what errors as shown above.

In [ ]:
# Display error
try:
    schema(df)
except SchemaError as e:
    print("Failed check:", e.check)
    print("\nInvalidated dataframe:\n", e.data)
    print("\nFailure cases:\n", e.failure_cases)

Catch the exception using try...except pattern to access the data and failure cases as attributes in `SchemaError` object.

## <a name="duplicate">Data Duplication
Question: How can we tell whether we have duplicate values in our dataset? 

In [ ]:
schema_duplicate = pa.DataFrameSchema({
    "Student ID" : pa.Column(int)
    },
    unique=["Student ID"])

In [ ]:
schema_duplicate.validate(df)

## <a name="exercise">Exercise

Given dataset based on customer feedback at a shopping mall. You need to perform data validation based on criteria below:
1. Customer ID must be unique.
2. Gender must be either "Male" or "Female".
3. The annual income must be in valid range(no negative numbers).
4. The range of spending score must be between 1 to 100.

In [ ]:
shop = pd.read_csv("../data/shopping.csv")

In [ ]:
shop.head()

In [ ]:
# show data types in each columns

In [ ]:
# Define schema

In [ ]:
# Validate schema

### <b>Now we should ask ourselves : What is the reason for these errors?
* Do we have an improperly defined schema?
* Do we expect to have negative values in our data?
* Why do we have negative values in **Age** column? Typo?
* Why do we have negative values in **Spending Score**? Is it because customer receive poor service?
* What should we do with our schema and our failing data points?